# Part 1

Somehow, a network packet got lost and ended up here. It's trying to follow a routing diagram (your puzzle input), but it's confused about where to go.

Its starting point is just off the top of the diagram. Lines (drawn with `|`, `-`, and `+`) show the path it needs to take, starting by going down onto the only line connected to the top of the diagram. It needs to follow this path until it reaches the end (located somewhere within the diagram) and stop there.

Sometimes, the lines cross over each other; in these cases, it needs to continue going the same direction, and only turn left or right when there's no other option. In addition, someone has left letters on the line; these also don't change its direction, but it can use them to keep track of where it's been. For example:

```
     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ 
```

Given this diagram, the packet needs to take the following path:

Starting at the only line touching the top of the diagram, it must go down, pass through A, and continue onward to the first +.
Travel right, up, and right, passing through B in the process.
Continue down (collecting C), right, and up (collecting D).
Finally, go all the way left through E and stopping at F.
Following the path to the end, the letters it sees on its path are `ABCDEF`.

The little packet looks up at you, hoping you can help it find the way. What letters will it see (in the order it would see them) if it follows the path? (The routing diagram is very wide; make sure you view it without line wrapping.)

In [25]:
from io import StringIO
from itertools import islice
from textwrap import dedent

import numpy as np

In [45]:
def process_path(rows):
    return tuple(tuple(c for c in row if c != '\n') for row in rows)

In [46]:
def _test_input():
    return process_path(StringIO(("""\
     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ 
                
""")))
TEST_INPUT = _test_input()

In [47]:
def _puzzle_input():
    with open('./inputs/day19/input.txt') as f:
        return process_path(f)
PUZZLE_INPUT = _puzzle_input()

In [12]:
def add_(x, y):
    return (x[0] + y[0], x[1] + y[1])

def get_(path, pos):
    return path[pos[0]][pos[1]]

In [68]:
move_map = {
    'down': (1, 0),
    'up': (-1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

opposites = {
    'up': 'down',
    'down': 'up',
    'left': 'right',
    'right': 'left'
}


def step(path, pos, dir_):
    while True:
        next_pos = add_(pos, move_map[dir_])
        char = get_(path, next_pos)
        if char != ' ':
            # continue straight
            pos = next_pos
        else:
            # make a turn
            for new_dir in (d for d in move_map if d not in (dir_, opposites[dir_])):
                next_pos = add_(pos, move_map[new_dir])
                char = get_(path, next_pos)
                if char != ' ':
                    pos = next_pos
                    dir_ = new_dir
                    break
            else:
                # end of the path
                break
        
        yield pos, dir_, char


def follow_path(path):
    start = (0, path[0].index('|'))
    dir_ = 'down'
    
    yield '|'  # include the first step
    
    for pos, dir_, char in step(path, start, dir_):
        yield char

In [69]:
def letters_in_path(path):
    yield from (c for c in follow_path(path) if c.isalpha())

In [70]:
result = ''.join(letters_in_path(TEST_INPUT))
assert result == 'ABCDEF', result

In [71]:
''.join(letters_in_path(PUZZLE_INPUT))

'SXPZDFJNRL'

# Part 2

The packet is curious how many steps it needs to go.

For example, using the same routing diagram from the example above...

```
     |          
     |  +--+    
     A  |  C    
 F---|--|-E---+ 
     |  |  |  D 
     +B-+  +--+ 
                
```

...the packet would go:

- 6 steps down (including the first line at the top of the diagram).
- 3 steps right.
- 4 steps up.
- 3 steps right.
- 4 steps down.
- 3 steps right.
- 2 steps up.
- 13 steps left (including the F it stops on).

This would result in a total of 38 steps.

How many steps does the packet need to go?

In [73]:
result = sum(1 for _ in follow_path(TEST_INPUT))
assert result == 38, result

In [74]:
sum(1 for _ in follow_path(PUZZLE_INPUT))

18126